# 🔍 SHAP Analysis: AQI Prediction Model Explainability

**Purpose:** Understand which features drive our AQI predictions using SHAP (SHapley Additive exPlanations)

**Model:** Random Forest Multi-Output Regressor (predicts 24h, 48h, 72h)

---

## 1. Setup and Imports

In [ ]:
import sys
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add parent directory to path
sys.path.append('..')

# Install SHAP if not available
try:
    import shap
    print(f"✅ SHAP version: {shap.__version__}")
except ImportError:
    print("Installing SHAP...")
    !pip install shap
    import shap

# Enable SHAP's JS visualizations in notebook
shap.initjs()

print("✅ All imports successful!")

## 2. Load Trained Model and Data

In [ ]:
# Load the trained multi-output model
model_path = "../models/best_model_multi_output_24h_48h_72h.pkl"

with open(model_path, 'rb') as f:
    model_data = pickle.load(f)

model = model_data['model']
feature_names = model_data['feature_names']
model_name = model_data.get('model_name', 'Unknown')
metrics = model_data.get('metrics', {})

print(f"✅ Model loaded: {model_name}")
print(f"📊 Features: {len(feature_names)}")
print(f"🎯 Model Type: Multi-Output (24h, 48h, 72h)")
print(f"")
print(f"📈 Model Performance:")
print(f"   Average RMSE: {metrics.get('rmse', 'N/A'):.2f}")
print(f"   Average R²: {metrics.get('r2', 'N/A'):.4f}")

In [ ]:
# Load feature data from MongoDB
from src.database import Database

db = Database()
features = db.get_features()
df = pd.DataFrame(features)

print(f"✅ Loaded {len(df)} feature records from MongoDB")

# Prepare X with only the features used by the model
available_features = [f for f in feature_names if f in df.columns]
X = df[available_features].dropna()

print(f"📊 Using {len(available_features)} features for SHAP analysis")
print(f"📊 Samples: {len(X)}")

## 3. Feature Importance (Built-in Random Forest)

Random Forest has built-in feature importance based on how much each feature reduces impurity across all trees.

In [ ]:
# Get feature importance from Random Forest
# For multi-output, we average importance across all outputs

importance = model.feature_importances_

# Create importance DataFrame
importance_df = pd.DataFrame({
    'Feature': available_features,
    'Importance': importance
}).sort_values('Importance', ascending=False)

# Display top 15 features
print("\n🏆 Top 15 Most Important Features:")
print("=" * 40)
for i, row in importance_df.head(15).iterrows():
    bar = "█" * int(row['Importance'] * 100)
    print(f"{row['Feature']:<25} {row['Importance']:.4f} {bar}")

In [ ]:
# Visualize Feature Importance
plt.figure(figsize=(12, 8))

top_n = 15
top_features = importance_df.head(top_n)

colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, top_n))[::-1]

plt.barh(range(top_n), top_features['Importance'], color=colors)
plt.yticks(range(top_n), top_features['Feature'])
plt.xlabel('Feature Importance (Mean Decrease in Impurity)')
plt.title('Top 15 Feature Importance - Random Forest Multi-Output Model', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()

plt.tight_layout()
plt.savefig('shap_charts/1_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Chart saved: shap_charts/1_feature_importance.png")

## 4. SHAP Analysis (for 24h Prediction)

SHAP values show how each feature contributes to individual predictions.

For multi-output, we'll analyze the 24h prediction (first output) as our primary horizon.

In [ ]:
# Create SHAP explainer for Random Forest
# We'll use a sample of data for efficiency

# Sample 100 data points for SHAP calculation (faster)
sample_size = min(100, len(X))
X_sample = X.sample(n=sample_size, random_state=42)

print(f"🔍 Computing SHAP values for {sample_size} samples...")
print("   (This may take a minute)")

# Create TreeExplainer (efficient for Random Forest)
explainer = shap.TreeExplainer(model)

# Calculate SHAP values
shap_values = explainer.shap_values(X_sample)

print("✅ SHAP values calculated!")
print(f"   Shape: {len(shap_values)} outputs × {shap_values[0].shape}")

In [ ]:
# SHAP Summary Plot for 24h Prediction (first output)
print("\n📊 SHAP Summary Plot - 24h Prediction")
print("   Each dot = one prediction")
print("   Color = feature value (red=high, blue=low)")
print("   X-axis = impact on prediction")

plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values[0], X_sample, feature_names=available_features, show=False)
plt.title('SHAP Summary Plot - 24h AQI Prediction', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('shap_charts/2_shap_summary_24h.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Chart saved: shap_charts/2_shap_summary_24h.png")

In [ ]:
# SHAP Bar Plot - Mean Absolute Impact
print("\n📊 SHAP Feature Importance (Mean |SHAP|)")

plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values[0], X_sample, feature_names=available_features, plot_type="bar", show=False)
plt.title('Mean |SHAP| Values - Feature Importance for 24h Prediction', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('shap_charts/3_shap_importance_bar.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Chart saved: shap_charts/3_shap_importance_bar.png")

## 5. Single Prediction Explanation (Waterfall Plot)

Let's explain ONE specific prediction to show how each feature pushed the prediction higher or lower.

In [ ]:
# Explain a single prediction
sample_idx = 0  # First sample

print(f"\n🔍 Explaining prediction for sample {sample_idx}:")
print(f"   Actual features: (first 5 shown)")
for feat in available_features[:5]:
    print(f"   - {feat}: {X_sample.iloc[sample_idx][feat]:.2f}")

# Get prediction
pred = model.predict(X_sample.iloc[[sample_idx]])[0]
print(f"\n   Prediction: 24h={pred[0]:.1f}, 48h={pred[1]:.1f}, 72h={pred[2]:.1f} PM2.5")

In [ ]:
# Waterfall plot for single prediction
print("\n📊 Waterfall Plot - How features affect this prediction")

plt.figure(figsize=(12, 8))
shap.plots.waterfall(shap.Explanation(
    values=shap_values[0][sample_idx],
    base_values=explainer.expected_value[0],
    data=X_sample.iloc[sample_idx],
    feature_names=available_features
), show=False)
plt.title(f'Waterfall Plot - Explaining 24h Prediction (Sample {sample_idx})', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('shap_charts/4_waterfall_explanation.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Chart saved: shap_charts/4_waterfall_explanation.png")

## 6. Feature Dependence Plot

Shows how the most important feature affects predictions across all samples.

In [ ]:
# Get top feature
top_feature = importance_df.iloc[0]['Feature']
print(f"\n📊 Dependence Plot for top feature: {top_feature}")

plt.figure(figsize=(10, 6))
shap.dependence_plot(
    top_feature, 
    shap_values[0], 
    X_sample,
    feature_names=available_features,
    show=False
)
plt.title(f'SHAP Dependence Plot: {top_feature}', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('shap_charts/5_dependence_plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Chart saved: shap_charts/5_dependence_plot.png")

## 7. Multi-Output Comparison

Compare feature importance across all three prediction horizons (24h, 48h, 72h).

In [ ]:
# Compare SHAP importance across horizons
horizon_names = ['24h', '48h', '72h']

# Calculate mean |SHAP| for each horizon
importance_by_horizon = {}
for i, horizon in enumerate(horizon_names):
    mean_shap = np.abs(shap_values[i]).mean(axis=0)
    importance_by_horizon[horizon] = mean_shap

# Create comparison DataFrame
comparison_df = pd.DataFrame(importance_by_horizon, index=available_features)
comparison_df['Average'] = comparison_df.mean(axis=1)
comparison_df = comparison_df.sort_values('Average', ascending=False).head(10)

print("\n📊 Top 10 Features Importance by Horizon:")
print(comparison_df.round(4))

In [ ]:
# Visualize comparison
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(comparison_df))
width = 0.25

bars1 = ax.bar(x - width, comparison_df['24h'], width, label='24h', color='#2ecc71')
bars2 = ax.bar(x, comparison_df['48h'], width, label='48h', color='#3498db')
bars3 = ax.bar(x + width, comparison_df['72h'], width, label='72h', color='#e74c3c')

ax.set_ylabel('Mean |SHAP| Value')
ax.set_title('Feature Importance Comparison Across Prediction Horizons', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df.index, rotation=45, ha='right')
ax.legend()

plt.tight_layout()
plt.savefig('shap_charts/6_horizon_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Chart saved: shap_charts/6_horizon_comparison.png")

## 8. Key Insights Summary

In [ ]:
print("\n" + "=" * 60)
print("🎯 KEY INSIGHTS FROM SHAP ANALYSIS")
print("=" * 60)

# Top 5 features
top5 = importance_df.head(5)
print("\n📊 Top 5 Most Important Features:")
for i, (_, row) in enumerate(top5.iterrows(), 1):
    print(f"   {i}. {row['Feature']} ({row['Importance']*100:.1f}%)")

# Weather features importance
weather_features = ['temp', 'humidity', 'pressure', 'wind_speed', 'clouds']
weather_importance = importance_df[importance_df['Feature'].isin(weather_features)]['Importance'].sum()
print(f"\n🌤️ Weather Features Total Importance: {weather_importance*100:.1f}%")

# Lag/Rolling features importance
temporal_features = [f for f in available_features if 'lag' in f or 'rolling' in f]
temporal_importance = importance_df[importance_df['Feature'].isin(temporal_features)]['Importance'].sum()
print(f"⏰ Temporal Features (Lag/Rolling) Importance: {temporal_importance*100:.1f}%")

# Pollutant features importance
pollutant_features = ['pm2_5', 'pm10', 'no2', 'o3', 'co', 'so2']
pollutant_importance = importance_df[importance_df['Feature'].isin(pollutant_features)]['Importance'].sum()
print(f"🏭 Pollutant Features Importance: {pollutant_importance*100:.1f}%")

print("\n" + "=" * 60)
print("✅ SHAP Analysis Complete!")
print("=" * 60)

---

## 📁 Generated Charts

| Chart | Description |
|-------|-------------|
| `1_feature_importance.png` | Random Forest built-in feature importance |
| `2_shap_summary_24h.png` | SHAP summary plot for 24h prediction |
| `3_shap_importance_bar.png` | Mean |SHAP| values as bar chart |
| `4_waterfall_explanation.png` | Single prediction breakdown |
| `5_dependence_plot.png` | Top feature's effect on predictions |
| `6_horizon_comparison.png` | Feature importance across 24h/48h/72h |

---

**Conclusion:** This analysis shows which features most strongly influence our AQI predictions, helping us understand and trust the model's decisions.